In [69]:
import psycopg2
import pandas as pd

In [70]:
conn = psycopg2.connect(database="neo_bank_1", user="neoflex_user",
    password="neoflex_user", host="localhost", port=5432)

In [77]:
''' Parse ft_balance_f.csv and push to neo_bank_1  '''
def push_ft_balance_f():
    df = pd.read_csv(
        filepath_or_buffer='../../../src/1/1/ft_balance_f.csv', 
        header='infer', 
        sep=';',
        usecols=range(1,5),
        parse_dates=['ON_DATE'],
        dayfirst=True
    )
    df.columns= df.columns.str.lower()
    
    insert_query = '''
        INSERT INTO ds.ft_balance_f (on_date, account_rk, currency_rk, balance_out) 
        VALUES %s
        ON CONFLICT (on_date, account_rk)  DO UPDATE 
            SET on_date = EXCLUDED.on_date, 
                account_rk = EXCLUDED.account_rk,
                currency_rk = EXCLUDED.currency_rk,
                balance_out = EXCLUDED.balance_out
    '''
    with conn:
        psycopg2.extras.execute_values (conn.cursor(), insert_query, df.values.tolist())

In [78]:
push_ft_balance_f()